# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [35]:
import torch
import torch.nn as nn

In [36]:
print(torch.backends.mps.is_available())

True


In [37]:
device = torch.device("cpu") # Defaults to CPU

# Example: Move a tensor or model to the MPS device
class TwoBranch(nn.Module):
    def __init__(self):
        super().__init__()
        model = nn.LSTM(input_size=3, hidden_size=16, batch_first=False, dropout=0.2).to(device)
        self.left = model
        self.right = model
        self.combine = nn.Linear(32, 32).to(device)
        nn.init.xavier_uniform_(model.weight_ih_l0)
        nn.init.xavier_uniform_(model.weight_hh_l0)
    def forward(self, x_left, x_right):
        _, (l_h, _) = self.left(x_left)
        _, (r_h, _) = self.right(x_right)

        l = l_h[-1]  # last layer, shape: (batch, hidden_size)
        r = r_h[-1]

        cat = torch.cat([l, r], dim=-1)  # shape: (batch, 10)
        return self.combine(cat)          # shape: (batch, 10) -> Linear(10,1) -> (batch,1)
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.branch = TwoBranch()
        self.rest = nn.Sequential(
            # nn.ReLU(),
            # nn.Linear(4, 4),
            # nn.ReLU(),
            # nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        ).to(device)

    def forward(self, x_left, x_right):
        x = self.branch(x_left, x_right)
        return self.rest(x)
model2=BigModel().to(device)



/Users/everettwilber/PyCharmMiscProject/.venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [38]:
"""
batch = 32
seq_len = 64

x_left  = torch.randn(seq_len, batch, 3, device=device)
x_right = torch.randn(seq_len, batch, 3, device=device)

# labels depend on problem:
y = torch.randn(batch, 1, device=device)  # regression
"""

'\nbatch = 32\nseq_len = 64\n\nx_left  = torch.randn(seq_len, batch, 3, device=device)\nx_right = torch.randn(seq_len, batch, 3, device=device)\n\n# labels depend on problem:\ny = torch.randn(batch, 1, device=device)  # regression\n'

In [39]:
import pandas as pd
import glob
files = glob.glob("data/*.json")
people = []
for file in files:
    df = pd.read_json(file)
    drawings = []
    for drawing in df.values:
        points = drawing[0][0]["points"]
        triples = [(point["x"], point["y"], point["time"]) for point in points]
        drawings.append(triples)
    people.append(drawings)


data = people


In [40]:
import json
from pathlib import Path

out_path = Path("output/data.json")

out_path.write_text(json.dumps(data, indent=2))

941031

In [41]:
out_path = Path("output/data.json")
loaded = json.load(out_path.open())

In [42]:
import numpy as np

training: list[list[list[tuple[int, int, int]]]] = loaded
pairedDeep = [[torch.tensor(drawing, device=device) for drawing in drawings] for drawings in training]
paired: list[torch.Tensor] = []
personIDs: list[int] = []
for personID, person in enumerate(pairedDeep):
    person = [drawing[:-1]-drawing[1:] for drawing in person]
    a = [personID] * len(person)
    personIDs.extend(a)
    paired.extend([((i[:, :] - i.min(dim=0, keepdim=True).values) / (
    (i.max(dim=0, keepdim=True).values - i.min(dim=0, keepdim=True).values)))*2-0.5 for i in person])
personIDs = np.array(personIDs)
lengths = torch.tensor([len(s) for s in paired])


def packPad(pairs: list[torch.Tensor], lengths: torch.Tensor, indeces):
    padded = nn.utils.rnn.pad_sequence([pairs[i].to(device) for i in indeces], batch_first=True).to(device)
    lengths = lengths[indeces]
    packed = nn.utils.rnn.pack_padded_sequence(
        padded,
        lengths,
        batch_first=True,
        enforce_sorted=False
    ).float().to(device)
    return packed



# Split at PERSON level to prevent any leakage
n_people = len(pairedDeep)
split_person_idx = int(n_people * 0.8)

# Get all indices for train people and test people
train_person_indices = []
test_person_indices = []

for person_id in range(n_people):
    person_mask = personIDs == person_id
    person_samples = np.where(person_mask)[0]

    if person_id < split_person_idx:
        train_person_indices.extend(person_samples)
    else:
        test_person_indices.extend(person_samples)

train_person_indices = np.array(train_person_indices)
test_person_indices = np.array(test_person_indices)

print(f"Training people: 0-{split_person_idx-1}")
print(f"Test people: {split_person_idx}-{n_people-1}")
print(f"Train samples: {len(train_person_indices)}")
print(f"Test samples: {len(test_person_indices)}")


def generate_balanced_pairs(indices, n_pairs, personIDs):
    """Generate 50/50 same-person vs different-person pairs"""
    left_indices = []
    right_indices = []

    n_same = n_pairs // 2
    n_diff = n_pairs - n_same

    # Same-person pairs
    for _ in range(n_same):
        idx = np.random.choice(indices)
        person_id = personIDs[idx]
        same_person_mask = personIDs[indices] == person_id
        same_person_indices = indices[same_person_mask]

        if len(same_person_indices) > 1:
            pair = np.random.choice(same_person_indices, size=2, replace=False)
            left_indices.append(pair[0])
            right_indices.append(pair[1])
        else:
            left_indices.append(idx)
            right_indices.append(idx)

    # Different-person pairs
    for _ in range(n_diff):
        idx1 = np.random.choice(indices)
        person1 = personIDs[idx1]
        diff_person_mask = personIDs[indices] != person1
        diff_person_indices = indices[diff_person_mask]

        if len(diff_person_indices) > 0:
            idx2 = np.random.choice(diff_person_indices)
            left_indices.append(idx1)
            right_indices.append(idx2)
        else:
            left_indices.append(idx1)
            right_indices.append(np.random.choice(indices))

    return np.array(left_indices), np.array(right_indices)


# Generate balanced pairs
n_train_pairs = len(train_person_indices)  # Reduce multiplier for stability
n_test_pairs = len(test_person_indices)

pLT, pRT = generate_balanced_pairs(train_person_indices, n_train_pairs, personIDs)
pLTest, pRTest = generate_balanced_pairs(test_person_indices, n_test_pairs, personIDs)

pairedNP = paired
lengthsNP = lengths
leftTrain = packPad(pairedNP, lengthsNP, pLT).to(device)
rightTrain = packPad(pairedNP, lengthsNP, pRT).to(device)
yTrain = (torch.tensor(personIDs[pLT] == personIDs[pRT])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))

leftTest = packPad(pairedNP, lengthsNP, pLTest).to(device)
rightTest = packPad(pairedNP, lengthsNP, pRTest).to(device)
yTest = (torch.tensor(personIDs[pLTest] == personIDs[pRTest])
         .reshape((-1, 1))
         .to(device, dtype=torch.float32))

print(f"\nTrain: {(yTrain == 1).sum().item() / len(yTrain) * 100:.1f}% same person")
print(f"Test:  {(yTest == 1).sum().item() / len(yTest) * 100:.1f}% same person")


Training people: 0-15
Test people: 16-20
Train samples: 214
Test samples: 63

Train: 50.0% same person
Test:  49.2% same person


In [43]:
str(((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean().item())

'0.4920634925365448'

In [44]:
model2.branch.left.all_weights

[[Parameter containing:
  tensor([[-1.0359e-01,  2.9640e-02, -1.9992e-01],
          [ 4.2961e-02,  2.3236e-01, -1.8219e-01],
          [-6.4471e-03,  1.1916e-01,  2.2864e-01],
          [ 2.9767e-01, -2.7202e-01,  2.9016e-02],
          [ 2.2845e-01, -1.8466e-01,  2.9212e-02],
          [-2.2355e-01, -1.8862e-01,  1.4822e-01],
          [ 5.6500e-02,  1.6762e-01,  1.0898e-02],
          [ 4.8944e-03,  6.9073e-02,  2.4441e-01],
          [-4.2027e-02, -4.2146e-03, -1.3859e-01],
          [ 9.7797e-02, -1.6945e-01,  1.6613e-01],
          [ 1.2677e-01, -2.3112e-03, -2.5536e-01],
          [-1.4962e-01,  1.3921e-01,  7.7057e-02],
          [ 1.7539e-02, -1.2712e-01, -6.9135e-02],
          [ 2.5398e-01,  5.8752e-02, -2.3297e-01],
          [-2.6583e-01, -1.2667e-01,  7.1329e-02],
          [-1.1717e-01, -2.3105e-01, -8.1025e-03],
          [-1.2629e-01, -2.9403e-01,  1.2356e-01],
          [-4.9178e-02,  2.5357e-01,  1.0162e-01],
          [-2.1155e-01,  2.5176e-01, -3.5774e-02],
       

In [45]:
epochs = 900
# BCELogits with L2 reg
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model2.parameters(), lr=0.001, weight_decay=1e-4)


In [46]:
annealer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
for epoch in range(epochs):
    pLT, pRT = generate_balanced_pairs(train_person_indices, n_train_pairs, personIDs)
    leftTrain = packPad(pairedNP, lengthsNP, pLT).to(device)
    rightTrain = packPad(pairedNP, lengthsNP, pRT).to(device)
    yTrain = (torch.tensor(personIDs[pLT] == personIDs[pRT])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))

    optimizer.zero_grad()
    epoch_loss = 0.0
    output = model2(leftTrain, rightTrain)
    loss = criterion(output, yTrain)

    loss.backward()
    optimizer.step()
    annealer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item()

    # average loss for the epoch

    if loss < 0.45:
        break
    if epoch % 1 == 0:
        print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")

model2

Epoch   0 – loss: 0.6936 - fPC 0.4921, 0.5000
Epoch   1 – loss: 0.6935 - fPC 0.4921, 0.5000
Epoch   2 – loss: 0.6932 - fPC 0.4921, 0.5000
Epoch   3 – loss: 0.6932 - fPC 0.4921, 0.5000
Epoch   4 – loss: 0.6932 - fPC 0.4921, 0.5047
Epoch   5 – loss: 0.6929 - fPC 0.5397, 0.5327
Epoch   6 – loss: 0.6928 - fPC 0.5079, 0.5654
Epoch   7 – loss: 0.6931 - fPC 0.5556, 0.5093
Epoch   8 – loss: 0.6933 - fPC 0.5079, 0.5140
Epoch   9 – loss: 0.6930 - fPC 0.5079, 0.5093
Epoch  10 – loss: 0.6928 - fPC 0.5079, 0.5047
Epoch  11 – loss: 0.6930 - fPC 0.5079, 0.5187
Epoch  12 – loss: 0.6928 - fPC 0.4921, 0.5000
Epoch  13 – loss: 0.6929 - fPC 0.5079, 0.5234
Epoch  14 – loss: 0.6934 - fPC 0.5079, 0.5234
Epoch  15 – loss: 0.6921 - fPC 0.5238, 0.5701
Epoch  16 – loss: 0.6939 - fPC 0.5079, 0.4813
Epoch  17 – loss: 0.6925 - fPC 0.5079, 0.5374
Epoch  18 – loss: 0.6927 - fPC 0.4921, 0.5047
Epoch  19 – loss: 0.6926 - fPC 0.4921, 0.5327
Epoch  20 – loss: 0.6928 - fPC 0.4921, 0.5187
Epoch  21 – loss: 0.6929 - fPC 0.5

BigModel(
  (branch): TwoBranch(
    (left): LSTM(3, 16, dropout=0.2)
    (right): LSTM(3, 16, dropout=0.2)
    (combine): Linear(in_features=32, out_features=32, bias=True)
  )
  (rest): Sequential(
    (0): ReLU()
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [47]:
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model2.parameters(), lr=0.001, weight_decay=2e-1)

In [48]:
annealer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=250)
for epoch in range(250):
    pLT, pRT = generate_balanced_pairs(train_person_indices, n_train_pairs*8, personIDs)
    leftTrain = packPad(pairedNP, lengthsNP, pLT).to(device)
    rightTrain = packPad(pairedNP, lengthsNP, pRT).to(device)
    yTrain = (torch.tensor(personIDs[pLT] == personIDs[pRT])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))


    optimizer.zero_grad()
    epoch_loss = 0.0
    output = model2(leftTrain, rightTrain)
    loss = criterion(output, yTrain)

    loss.backward()
    optimizer.step()
    annealer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item()

    # average loss for the epoch
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")



print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")

Epoch   0 – loss: 0.4754 - fPC 0.8413, 0.7950
Epoch  10 – loss: 0.4732 - fPC 0.7937, 0.8072
Epoch  20 – loss: 0.4373 - fPC 0.8095, 0.8224
Epoch  30 – loss: 0.4330 - fPC 0.8095, 0.8254
Epoch  40 – loss: 0.4108 - fPC 0.8095, 0.8411
Epoch  50 – loss: 0.4311 - fPC 0.8095, 0.8265
Epoch  60 – loss: 0.4082 - fPC 0.7937, 0.8335
Epoch  70 – loss: 0.3892 - fPC 0.7619, 0.8452
Epoch  80 – loss: 0.3818 - fPC 0.7619, 0.8563
Epoch  90 – loss: 0.3939 - fPC 0.7619, 0.8347
Epoch 100 – loss: 0.3654 - fPC 0.7619, 0.8563
Epoch 110 – loss: 0.3653 - fPC 0.7619, 0.8493
Epoch 120 – loss: 0.3790 - fPC 0.7778, 0.8516
Epoch 130 – loss: 0.3821 - fPC 0.7619, 0.8511
Epoch 140 – loss: 0.3863 - fPC 0.7778, 0.8452
Epoch 150 – loss: 0.3754 - fPC 0.7778, 0.8446
Epoch 160 – loss: 0.3662 - fPC 0.7619, 0.8610
Epoch 170 – loss: 0.3767 - fPC 0.7619, 0.8528
Epoch 180 – loss: 0.3583 - fPC 0.7778, 0.8616
Epoch 190 – loss: 0.3672 - fPC 0.7778, 0.8540
Epoch 200 – loss: 0.3725 - fPC 0.7778, 0.8522
Epoch 210 – loss: 0.3741 - fPC 0.7

In [49]:
model = model2

In [50]:
ckpt = {
    'epoch'         : epoch,                     # current epoch number
    'model_state'   : model.state_dict(),        # model parameters
    'optimizer_state': optimizer.state_dict(),   # optimizer internals
    'scheduler_state': annealer.state_dict() if annealer else None,
}
torch.save(ckpt, 'model.pt')
torch.save(model.state_dict(), 'model_weights.pt')


In [51]:
import torch

def confusion_rates(yhat: torch.Tensor, y: torch.Tensor):
    tp = (yhat == 1) & (y == 1)
    tn = (yhat == 0) & (y == 0)
    fp = (yhat == 1) & (y == 0)
    fn = (yhat == 0) & (y == 1)

    TP = int(tp.sum().item())
    TN = int(tn.sum().item())
    FP = int(fp.sum().item())
    FN = int(fn.sum().item())

    eps = 1e-12
    TPR = TP / (TP + FN + eps)   # recall / sensitivity
    TNR = TN / (TN + FP + eps)   # specificity
    FPR = FP / (FP + TN + eps)   # 1 - specificity
    FNR = FN / (FN + TP + eps)   # 1 - recall

    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN,
            "TPR": TPR, "TNR": TNR, "FPR": FPR, "FNR": FNR}

def metrics(model, leftTest, rightTest, yTest):
    yhat = model(leftTest, rightTest) > 0
    y = yTest
    return confusion_rates(yhat, y)

res = metrics(model2, leftTest, rightTest, yTest)

print(f"True Positive Rate (Recall) : {res['TPR']:.4f}; TP : {res['TP']}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}; TN : {res['TN']}")
print(f"False Positive Rate         : {res['FPR']:.4f}; FP : {res['FP']}")
print(f"False Negative Rate         : {res['FNR']:.4f}; FN : {res['FN']}")


True Positive Rate (Recall) : 0.8387; TP : 26
True Negative Rate (Spec.)  : 0.7187; TN : 23
False Positive Rate         : 0.2812; FP : 9
False Negative Rate         : 0.1613; FN : 5


In [52]:
weights = torch.load('model_weights.pt', map_location=device)
model_new = BigModel()          # the same class you used during training
model_new.load_state_dict(weights)
model_new.to(device)

res = metrics(model_new, leftTest, rightTest, yTest)

print(f"True Positive Rate (Recall) : {res['TPR']:.4f}; TP : {res['TP']}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}; TN : {res['TN']}")
print(f"False Positive Rate         : {res['FPR']:.4f}; FP : {res['FP']}")
print(f"False Negative Rate         : {res['FNR']:.4f}; FN : {res['FN']}")

True Positive Rate (Recall) : 0.8387; TP : 26
True Negative Rate (Spec.)  : 0.7187; TN : 23
False Positive Rate         : 0.2812; FP : 9
False Negative Rate         : 0.1613; FN : 5


/Users/everettwilber/PyCharmMiscProject/.venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
